   # Within-fire variability 
   
   Workflow to extract pixels from imagery and boostrap samples to get median and mean for each image.

In [ ]:
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np
import dask
%matplotlib inline
%tb


In [28]:
# required functions
%run processing_functions.py
%run ../pixel_to_df.py

## Test workflow on single fire

What is the distribution of pixel counts in  defol/non-defoliated between and within fires?


Goals: 
1) create reference distribution from one of defoliation / non-defoiated halves
2) from the opposite half, subsample to match sample size of reference. 
3) boostrap resample to get CI and distrbution of values. 
4) compare distribution

Steps:
1) get coordinates and return them as a dataframe with x and y
2) add column to df which says whether it is defoliated or non-defoliated (binary)
3) extract pixel values? 
4) randomly select a subsample of pixels from df, get values...etc.?

* turn into 4d array with time, band, x, y - band would be nbr and defoliated (1,0)


NOTES:
- values to be standardized when in DF format after pixel extraction

In [134]:
# read in data and get coordinates
ds = rx.open_rasterio("/Users/jgoldman/Desktop/chp3/clipped/nbr/non-defoliated/COC16_1997_1038.nc", masked=True)

lat = ds.coords['y'].values
lon = ds.coords['x'].values


In [59]:
# another option

time, lat2, lon2 = ds.indexes.values()
time
ds.count()

ds2= ds.isel(time=2)
df = ds2.to_dataframe()
df

time  crs  nbr
y         x                           
49.943680 -80.599497  1998.0    0  NaN
          -80.599228  1998.0    0  NaN
          -80.598958  1998.0    0  NaN
          -80.598689  1998.0    0  NaN
          -80.598419  1998.0    0  NaN
...                      ...  ...  ...
49.853669 -80.490083  1998.0    0  NaN
          -80.489813  1998.0    0  NaN
          -80.489544  1998.0    0  NaN
          -80.489274  1998.0    0  NaN
          -80.489005  1998.0    0  NaN

[137685 rows x 3 columns]

In [192]:
# random sample for a given year.
# for each sample take the coordinates and get values across df.


def random_sample(da,
                  n, 
                  dim_slice,
                  keep_cols=True):
    """
    take a data array, turn to df and get random sample
    da = xarray data array with n dimensions
    n = number of samples
    dim_slice = slice along certain time dimension
    keep_cols = 

    """
    # slice along dimension
    da2= da.isel(time=dim_slice)
    #to dataframe
    df = da2.to_dataframe()

    # sample location
    samples = []
    no_of_points = n
    # random sample of n points
    sample_loc = df.dropna().sample(n=int(round(no_of_points)))
    samples.append(sample_loc)
    #join back into single datafame
    all_samples = pd.concat([samples[i] for i in range(0,len(samples))])
    
    # x and y to lat and lon column
    all_samples.reset_index(inplace = True)
    all_samples = all_samples.rename(columns={'y': 'Lat', 'x': 'Long'}) 
    
    if keep_cols==False:
        #remove crs, nbr and time cols
        all_samples = all_samples.drop(columns =['crs', 'time', 'nbr'])
     # for each lat/long select pixel
    pix_samples = []
    for index, row in all_samples.iterrows():
        x = row['Long']
        y = row['Lat']
        df2 = da.to_dataframe()
        df2 = df2.dropna()
        df2 = std_nbr(df2)
        df2 = df2.loc[(df2['y'] == y) & (df2['x'] == x)]
        
        # add id row to df2
        # add add row that goes preNBR - nbr 10
        
        
        pix_samples.append(df2)

    #join back into single datafame
    all_samples = pd.concat([pix_samples[i] for i in range(0,len(pix_samples))])

    all_samples
  
    #remove crs
    all_samples = all_samples.drop(columns =['crs'])
    
    return(all_samples)

In [191]:
df = random_sample(ds, 1000, 1, False)
df

,year,y,x,crs,nbr
19072,1996,49.889242,-80.559343,0,0.642078
49254,1997,49.889242,-80.559343,0,0.117755
79436,1998,49.889242,-80.559343,0,0.143440
109618,1999,49.889242,-80.559343,0,0.169125
139800,2000,49.889242,-80.559343,0,0.194810
...,...,...,...,...,...
216736,2003,49.919425,-80.573087,0,0.306862
246918,2004,49.919425,-80.573087,0,0.318674
277100,2005,49.919425,-80.573087,0,0.330486
307282,2006,49.919425,-80.573087,0,0.342298
